In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/landslide/train/Non-Landslides/fyb209.png
/kaggle/input/landslide/train/Non-Landslides/fyb157.png
/kaggle/input/landslide/train/Non-Landslides/dwnpl05139.png
/kaggle/input/landslide/train/Non-Landslides/dnypl05025.png
/kaggle/input/landslide/train/Non-Landslides/fyb191.png
/kaggle/input/landslide/train/Non-Landslides/dnypl05005.png
/kaggle/input/landslide/train/Non-Landslides/fyb62.png
/kaggle/input/landslide/train/Non-Landslides/fyb438.png
/kaggle/input/landslide/train/Non-Landslides/fyb479.png
/kaggle/input/landslide/train/Non-Landslides/ddfgf10167.png
/kaggle/input/landslide/train/Non-Landslides/dhzgf10146.png
/kaggle/input/landslide/train/Non-Landslides/fyb66.png
/kaggle/input/landslide/train/Non-Landslides/fyb314.png
/kaggle/input/landslide/train/Non-Landslides/dnypl05028.png
/kaggle/input/landslide/train/Non-Landslides/fyb192.png
/kaggle/input/landslide/train/Non-Landslides/fyb83.png
/kaggle/input/landslide/train/Non-Landslides/dzjwv05278.png
/kaggle/input/landslide

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Conv2D, MaxPooling2D, Conv2DTranspose, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import f1_score, precision_score, recall_score

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [3]:
train_data = train_datagen.flow_from_directory(
    '/kaggle/input/landslide/train',  # Parent directory containing Landslide and Non-Landslides folders
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 1272 images belonging to 2 classes.


In [4]:
val_data = train_datagen.flow_from_directory(
    '/kaggle/input/landslide/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 318 images belonging to 2 classes.


In [5]:
# Custom metrics
def f1_m(y_true, y_pred):
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float'), axis=0)
    tn = tf.reduce_sum(tf.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), 'float'), axis=0)

    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return tf.reduce_mean(f1)

def precision_m(y_true, y_pred):
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float'), axis=0)
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, 'float'), axis=0)
    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    return tf.reduce_mean(precision)

def recall_m(y_true, y_pred):
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float'), axis=0)
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), 'float'), axis=0)
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    return tf.reduce_mean(recall)

In [8]:
# Define the U-Net++ model function
def unet_plus_plus_model(filtersFirstLayer, lr, input_size=(224, 224, 3)):
    inputs = Input(input_size)
    nb_filter = [32, 64, 128, 256, 512]
    s = Lambda(lambda x: x / 255)(inputs)
    c1 = Conv2D(filtersFirstLayer, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.5)(c1)
    c1 = Conv2D(filtersFirstLayer, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    # U-Net++ structure
    # ... Continue the architecture as given in your code
    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.5) (c2)
    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
    c2 = Dropout(0.5) (c2)
    p2 = MaxPooling2D((2, 2), strides=(2, 2)) (c2)
    
    up1_2 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up12', padding='same')(c2)
    conv1_2 = concatenate([up1_2, c1], name='merge12', axis=3)
    c3 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv1_2)
    c3 = Dropout(0.5) (c3)
    c3 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
    c3 = Dropout(0.5) (c3)

    conv3_1 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
    conv3_1 = Dropout(0.5) (conv3_1)
    conv3_1 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv3_1)
    conv3_1 = Dropout(0.5) (conv3_1)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_1)

    up2_2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up22', padding='same')(conv3_1)
    conv2_2 = concatenate([up2_2, c2], name='merge22', axis=3) #x10
    conv2_2 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (conv2_2)
    conv2_2 = Dropout(0.5) (conv2_2)
    conv2_2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv2_2)
    conv2_2 = Dropout(0.5) (conv2_2)

    up1_3 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up13', padding='same')(conv2_2)
    conv1_3 = concatenate([up1_3, c1, c3], name='merge13', axis=3)
    conv1_3 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv1_3)
    conv1_3 = Dropout(0.5) (conv1_3)
    conv1_3 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv1_3)
    conv1_3 = Dropout(0.5) (conv1_3)

    conv4_1 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (pool3)
    conv4_1 = Dropout(0.5) (conv4_1)
    conv4_1 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv4_1)
    conv4_1 = Dropout(0.5) (conv4_1)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4_1)

    up3_2 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up32', padding='same')(conv4_1)
    conv3_2 = concatenate([up3_2, conv3_1], name='merge32', axis=3) #x20
    conv3_2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv3_2)
    conv3_2 = Dropout(0.5) (conv3_2)
    conv3_2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv3_2)
    conv3_2 = Dropout(0.5) (conv3_2)

    up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up23', padding='same')(conv3_2)
    conv2_3 = concatenate([up2_3, c2, conv2_2], name='merge23', axis=3)
    conv2_3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv2_3)
    conv2_3 = Dropout(0.5) (conv2_3)
    conv2_3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv2_3)
    conv2_3 = Dropout(0.5) (conv2_3)

    up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up14', padding='same')(conv2_3)
    conv1_4 = concatenate([up1_4, c1, c3, conv1_3], name='merge14', axis=3)
    conv1_4 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv1_4)
    conv1_4 = Dropout(0.5) (conv1_4)
    conv1_4 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv1_4)
    conv1_4 = Dropout(0.5) (conv1_4)

    conv5_1 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (pool4)
    conv5_1 = Dropout(0.5) (conv5_1)
    conv5_1 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv5_1)
    conv5_1 = Dropout(0.5) (conv5_1)

    up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), name='up42', padding='same')(conv5_1)
    conv4_2 = concatenate([up4_2, conv4_1], name='merge42', axis=3) #x30
    conv4_2 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv4_2)
    conv4_2 = Dropout(0.5) (conv4_2)
    conv4_2 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv4_2)
    conv4_2 = Dropout(0.5) (conv4_2)

    up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up33', padding='same')(conv4_2)
    conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33', axis=3)
    conv3_3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv3_3)
    conv3_3 = Dropout(0.5) (conv3_3)
    conv3_3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv3_3)
    conv3_3 = Dropout(0.5) (conv3_3)

    up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up24', padding='same')(conv3_3)
    conv2_4 = concatenate([up2_4, c2, conv2_2, conv2_3], name='merge24', axis=3)
    conv2_4 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv2_4)
    conv2_4 = Dropout(0.5) (conv2_4)
    conv2_4 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv2_4)
    conv2_4 = Dropout(0.5) (conv2_4)

    up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up15', padding='same')(conv2_4)
    conv1_5 = concatenate([up1_5, c1, c3, conv1_3, conv1_4], name='merge15', axis=3)
    conv1_5 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv1_5)
    conv1_5 = Dropout(0.5) (conv1_5)
    conv1_5 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (conv1_5)
    conv1_5 = Dropout(0.5) (conv1_5)
    # Final output layer for classification
    x = tf.keras.layers.GlobalAveragePooling2D()(conv1_5)  # Pooling to get a single output per image
    nestnet_output_4 = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Compile the model for classification
    model = Model(inputs=[inputs], outputs=[nestnet_output_4])
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy', f1_m, precision_m, recall_m])
    return model
print("Done")

Done


In [9]:
# Instantiate the model
filtersFirstLayer = 32
learning_rate = 0.001
input_size = (224, 224, 3)
model = unet_plus_plus_model(filtersFirstLayer, learning_rate, input_size)

# Fit the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20
)

# Evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(val_data)
print(f"Loss: {loss}, Accuracy: {accuracy}, F1 Score: {f1_score}, Precision: {precision}, Recall: {recall}")

Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1731949895.834344     108 service.cc:145] XLA service 0x7fa200002940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731949895.834425     108 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2024-11-18 17:12:39.363523: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[32,96,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,96,224,224]{3,2,1,0}, f32[32,32,224,224]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFil

 2/40 ━━━━━━━━━━━━━━━━━━━━ 20s 531ms/step - accuracy: 0.6328 - f1_m: 0.6328 - loss: 0.9361 - precision_m: 0.6328 - recall_m: 0.6328   

2024-11-18 17:14:08.759793: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[32,96,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[24,96,224,224]{3,2,1,0}, f32[24,32,224,224]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-11-18 17:14:09.043614: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.284078652s
Trying algorithm eng0{} for conv (f32[32,96,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[24,96,224,224]{3,2,1,0}, f32[24,32,224,224]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_con

40/40 ━━━━━━━━━━━━━━━━━━━━ 242s 3s/step - accuracy: 0.5251 - f1_m: 0.3192 - loss: 0.7746 - precision_m: 0.3192 - recall_m: 0.3192 - val_accuracy: 0.5157 - val_f1_m: 0.5163 - val_loss: 0.6930 - val_precision_m: 0.5163 - val_recall_m: 0.5163
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 623ms/step - accuracy: 0.5344 - f1_m: 0.5344 - loss: 0.6928 - precision_m: 0.5344 - recall_m: 0.5344 - val_accuracy: 0.5157 - val_f1_m: 0.5156 - val_loss: 0.6929 - val_precision_m: 0.5156 - val_recall_m: 0.5156
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 625ms/step - accuracy: 0.5313 - f1_m: 0.5308 - loss: 0.6925 - precision_m: 0.5308 - recall_m: 0.5308 - val_accuracy: 0.5157 - val_f1_m: 0.5154 - val_loss: 0.6928 - val_precision_m: 0.5154 - val_recall_m: 0.5154
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 624ms/step - accuracy: 0.5042 - f1_m: 0.5059 - loss: 0.6932 - precision_m: 0.5059 - recall_m: 0.5059 - val_accuracy: 0.5157 - val_f1_m: 0.5148 - val_loss: 0.6927 - val_precision_m: 0.5148 - val_recall_m: 0.5148
E

In [15]:
model.evaluate(val_data)

10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 501ms/step - accuracy: 0.5195 - f1_m: 0.5195 - loss: 0.7226 - precision_m: 0.5195 - recall_m: 0.5195


[0.7247385382652283,
 0.5157232880592346,
 0.51583331823349,
 0.51583331823349,
 0.51583331823349]